In [ ]:
"""
Packages
"""

using Pkg; Pkg.activate(joinpath(@__DIR__, ".."))
using LinearAlgebra
using HybridRobotDynamics
using SiLQR

In [ ]:
"""
2D Double Integrator Model
"""

# System dimensions
nx = 4
nu = 2

# Dynamics ODE
ẋ = (x,u) -> [x[3:4]; u]
mI = HybridMode(ẋ)

# Transitions and modes
transitions = Dict{Symbol, Transition}(Dict())
modes = Dict(:nominal => mI)

# Define the system
system = HybridSystem(nx, nu, transitions, modes)

nothing

In [ ]:
"""
Solver Setup
"""

# Bouncing ball with thrust model
#system = bouncing_ball()

# Stage and terminal costs
Q = 1e-4 * diagm([1.0, 1.0, 0.1, 0.1])
R = 1e-6 * I(system.nu)
Qf = 1e2 * Q
stage(x, u) = x'*Q*x + u'*R*u
terminal(x) = x'*Qf*x

# RK4 integrator
rk4 = ExplicitIntegrator(:rk4)

# Problem parameters
N = 50
Δt = 0.1
params = ProblemParameters(system, stage, terminal, rk4, N, Δt)

# Reference trajectory and initial conditions
xref = [10.0; 10.0; 0.0; 0.0]
uref = zeros(system.nu)
params.xrefs = [xref for k = 1:N]
params.urefs = [uref for k = 1:(N-1)]
params.x0 = [0.0, 0.0, 0.0, 0.0]
params.mI = :nominal

nothing

In [ ]:
"""
Solve using SiLQR
"""

# Solve
terms = ProblemTerms(params)
SiLQR_solve!(terms, params)

# Visualize states
xs = reduce(vcat, terms.sol.xs)
plot_2d_states(N, system.nx, (1,2), xs)

nothing